# Introducción

La idea es scrapear hoteles y departamentos de una página de rentas venezolana. La idea surgió de leer [este proyecto](https://www.workana.com/job/scraping-a-pagina-web?ref=projects_1) en una página de trabajos freelance. Excepto que decidí cambiarla un poco.

# Extracción (scrapeo) de la información

In [3]:
import requests
from bs4 import BeautifulSoup

In [4]:
base_url = "https://rentahouse.com.ve/"

In [5]:
# Consigue los links a cada publicacion dentro de cada pagina de una seccion (casas por defecto)
def get_links(base_url, page=1, casa=True):
    if casa:
        url = base_url + "casa.html"
        payload = { 'page' : page, 'orderBy' : 'entryTimestamp desc', 'propertyTypeFromSlug' : 'Casa' }
    else:
        url = base_url + "apartamento.html"
        payload = { 'page' : page, 'orderBy' : 'entryTimestamp desc', 'propertyTypeFromSlug' : 'Apartamento' }
    
    req = requests.get(url, params=payload)
    soup = BeautifulSoup(req.text, "html.parser")
    homes = soup.find_all("div", class_='property-list')

    res = []
    for home in homes:
        res.append(home.a['href'])
    
    return res

In [6]:
links = get_links(base_url)

In [7]:
# Extrae la informacion (sucia) de una publicacion
def get_info_from_house(url):
    req = requests.get(url)
    soup = BeautifulSoup(req.text, "html.parser")

    info = soup.find('div', class_="propertyInfo").find('div', class_="row")
    data = {}

    data['Price'] = info.find('div', class_='price')['title']

    general_info = info.find('ul', class_="property-detailes-list").find_all('li')

    for item in general_info:
        spans = item.find_all('span')
        data[spans[0].text] = spans[1].text

    location = info.find_all('div', class_='DescripcionGeneral')[-1].find_all('li')

    for item in location:
        spans = item.find_all('span')
        data[spans[0].text] = spans[1].text

    return data

In [8]:
get_info_from_house(links[0])

{'Price': '$498,000 USD',
 'Codígo RAH:': 'VE 23-26224',
 'Tipo de Propiedad:': 'Casa',
 'Estilo:': '1 Nivel',
 'Área Privada:': '948 m2\n\n',
 'Terreno:': '948 m2\n\n',
 'Estado Del Inmueble:': 'Usado',
 'Dormitorios:': '5',
 'Total Baños:': '8',
 'Baños Completos:': '8',
 'Tipo De Estacionamiento:': 'Cubierto',
 'Puestos De Estacionamiento:': '10',
 'Amoblado:': 'Si',
 '✅Dormitorio De Servicio:': 'Si',
 'Calle:': 'Publica',
 'País: ': 'Venezuela',
 'Estado: ': 'Distrito Metropolitano',
 'Ciudad: ': 'Caracas',
 'Urbanización: ': 'Lomas del Club Hipico'}

In [13]:
# Consigue todas las publicaciones desde la primer pagina hasta la numero pages inclusive y las devuelve como lista
def get_house_links(pages, casa=True):
    homes = []
    for i in range(1,pages+1):
        print(f'Scrapeando pagina {i}')
        homes += get_links(base_url, page=i, casa=casa)
    
    return homes

In [10]:
house_links = get_house_links(30)

In [12]:
department_links = get_house_links(30, casa=False)

In [20]:
import sys
# Scrapea todas las casas de la lista conseguida por get_house_links
def scrape_homes(homes_list):
    homes = []
    count = 0
    for home in homes_list:
        homes.append(get_info_from_house(home))
        count += 1
        sys.stdout.write(f"\rElementos screapeados: {count}/{len(homes_list)}")

    return homes

In [21]:
houses = scrape_homes(house_links)

Elementos screapeados: 360/360

In [23]:
departments = scrape_homes(department_links)

Elementos screapeados: 360/360

# Limpieza y transformaciones con pandas

Ahora queda limpiar la información de cada casa, para esto cargo los datos obtenidos en dataframes de pandas.

In [25]:
import pandas as pd

In [151]:
houses_df = pd.DataFrame(houses)

In [152]:
houses_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 360 entries, 0 to 359
Data columns (total 20 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   Price                        360 non-null    object
 1   Codígo RAH:                  360 non-null    object
 2   Tipo de Propiedad:           360 non-null    object
 3   Estilo:                      360 non-null    object
 4   Área Privada:                360 non-null    object
 5   Terreno:                     360 non-null    object
 6   Estado Del Inmueble:         360 non-null    object
 7   Dormitorios:                 360 non-null    object
 8   Total Baños:                 360 non-null    object
 9   Baños Completos:             360 non-null    object
 10  Tipo De Estacionamiento:     360 non-null    object
 11  Puestos De Estacionamiento:  306 non-null    object
 12  Amoblado:                    360 non-null    object
 13  ✅Dormitorio De Servicio:     149 no

De la celda de arriba se puede observar que todas las columnas son strings, y que tienen nombres "feos". Para solucionar esto primero voy a renombrar las columnas y luego revisar sus tipos.

In [188]:
def rename_columns(df):
    new_col_names = {}
    for col in df.columns:
        old_col = str(col)
        if old_col.endswith(':') or old_col.endswith(': '):
            new_col = old_col.split(':')[0]
            new_col_names[old_col] = new_col

    df.rename(columns=new_col_names, inplace=True)
    df.rename(columns = {'✅Dormitorio De Servicio' : 'Dormitorio De Servicio'}, inplace=True)


In [154]:
rename_columns(houses_df)

In [155]:
departments_df = pd.DataFrame(departments)
rename_columns(departments_df)
departments_df.columns

Index(['Price', 'Codígo RAH', 'Tipo de Propiedad', 'Estilo', 'Área Privada',
       'Estado Del Inmueble', 'Dormitorios', 'Total Baños', 'Baños Completos',
       'Tipo De Estacionamiento', 'Puestos De Estacionamiento', 'Amoblado',
       'Dormitorio De Servicio', 'País', 'Estado', 'Ciudad', 'Urbanización',
       'Medios Baños'],
      dtype='object')

In [156]:
# Reviso por las dudas que de hecho todos los precios sean "$<numero> USD"
houses_df['Price'].apply(lambda x: not str(x).endswith('USD')).any()

False

In [157]:
def fix_prices(df):
    if not df['Price'].apply(lambda x: not str(x).endswith('USD')).any():
        df['Price'] = df['Price'].str.replace(r'\D+','', regex=True)
    df['Price'] = df['Price'].astype(int)
        

In [158]:
fix_prices(houses_df)
fix_prices(departments_df)

In [159]:
houses_df.head()

,Price,Codígo RAH,Tipo de Propiedad,Estilo,Área Privada,Terreno,Estado Del Inmueble,Dormitorios,Total Baños,Baños Completos,Tipo De Estacionamiento,Puestos De Estacionamiento,Amoblado,Dormitorio De Servicio,Calle,País,Estado,Ciudad,Urbanización,Medios Baños
0,498000,VE 23-26224,Casa,1 Nivel,948 m2\n\n,948 m2\n\n,Usado,5,8,8,Cubierto,10,Si,Si,Publica,Venezuela,Distrito Metropolitano,Caracas,Lomas del Club Hipico,NaN
1,240000,VE 23-26220,Casa,Multipes Niveles,307 m2\n\n,704 m2\n\n,Usado,5,6,5,Descubierto,4,No,Si,Cerrada con Vigilancia,Venezuela,Distrito Metropolitano,Caracas,El Marques,1
2,580000,VE 23-26215,Casa,1 Nivel,380 m2\n\n,1251 m2\n\n,Usado,5,6,5,Cubierto,5,No,Si,Cerrada con Vigilancia,Venezuela,Distrito Metropolitano,Caracas,Prados del Este,1
3,380000,VE 23-26209,Casa,Duplex,580 m2\n\n,504 m2\n\n,Usado,4,5,4,Cubierto,3,No,Si,Cerrada con Vigilancia,Venezuela,Distrito Metropolitano,Caracas,Sorocaima,1
4,270000,VE 23-26196,Casa,Multipes Niveles,400 m2\n\n,617 m2\n\n,Usado,4,3,3,Cubierto,5,No,Si,Cerrada con Vigilancia,Venezuela,Distrito Metropolitano,Caracas,Lomas del Halcon,NaN


In [160]:
def fix_numeric_columns(df):
    num_cols = ['Dormitorios', 'Total Baños', 'Baños Completos', 'Puestos De Estacionamiento', 'Medios Baños']
    for col in num_cols:
        df[col] = pd.to_numeric(df[col])

In [161]:
fix_numeric_columns(houses_df)
fix_numeric_columns(departments_df)

In [162]:
print("Columnas en comun: ", list(set(houses_df.columns) & set(departments_df.columns)))
print("Columnas en casas pero no depas: ",list(set(houses_df.columns) - set(departments_df.columns)))

Columnas en comun:  ['Urbanización', 'Price', 'Medios Baños', 'Tipo De Estacionamiento', 'Codígo RAH', 'Estilo', 'País', 'Dormitorio De Servicio', 'Amoblado', 'Ciudad', 'Puestos De Estacionamiento', 'Tipo de Propiedad', 'Dormitorios', 'Baños Completos', 'Área Privada', 'Estado', 'Total Baños', 'Estado Del Inmueble']
Columnas en casas pero no depas:  ['Calle', 'Terreno']


In [163]:
def fix_areas(df, cols):
    for col in cols:
        df[col] = df[col].str.replace(' m2\n\n', '')
        df[col] = pd.to_numeric(df[col])

In [164]:
fix_areas(houses_df, ['Terreno', 'Área Privada'])
fix_areas(departments_df, ['Área Privada'])

In [165]:
houses_df['Medios Baños'].value_counts(dropna=False)

NaN    221
1.0    104
0.0     16
2.0     13
3.0      5
6.0      1
Name: Medios Baños, dtype: int64

In [166]:
# Asumo que un medio baño nulo significa que tiene 0 medios baños
houses_df['Medios Baños'] = houses_df['Medios Baños'].fillna(0)
departments_df['Medios Baños'] = departments_df['Medios Baños'].fillna(0)

In [167]:
houses_df['Dormitorio De Servicio'].value_counts(dropna=False)

NaN    211
Si     149
Name: Dormitorio De Servicio, dtype: int64

In [168]:
# Reemplazo nulos por no
houses_df['Dormitorio De Servicio'] = houses_df['Dormitorio De Servicio'].fillna('No')
departments_df['Dormitorio De Servicio'] = departments_df['Dormitorio De Servicio'].fillna('No')

In [169]:
houses_df['Puestos De Estacionamiento'].value_counts(dropna=False)

2.0     68
3.0     60
4.0     59
NaN     54
5.0     38
1.0     31
6.0     17
8.0     10
10.0     8
7.0      6
12.0     4
0.0      2
15.0     2
20.0     1
Name: Puestos De Estacionamiento, dtype: int64

In [171]:
houses_df[(houses_df['Tipo De Estacionamiento'] != 'Ninguno') & (houses_df['Puestos De Estacionamiento'].isnull())][['Tipo De Estacionamiento', 'Puestos De Estacionamiento']].head()

,Tipo De Estacionamiento,Puestos De Estacionamiento
9,Cubierto,NaN
10,Cubierto,NaN
28,Cubierto,NaN
29,Cubierto,NaN
30,Descubierto,NaN


Inicialmente, crei que un puesto de estacionamiento nulo quería decir que no había puestos de estacionamiento, pero esto es incorrecto ya que hay publicaciones con tipo de estacionamiento distinto de "ninguno" pero con puestos de estacionamiento nulo. Debido a esto, no reemplace los nulos y los deje como están.

# Exportación a CSV

Ahora con los dataframes limpiados, los exporto a csv para persistirlos.

In [177]:
houses_df.rename(columns={'Price':'Precio'}, inplace=True)
departments_df.rename(columns={'Price':'Precio'}, inplace=True)

In [183]:
houses_df.to_csv('Casas_venezuela.csv', index=False)
departments_df.to_csv('Departamentos_venezuela.csv', index=False)